In [1]:
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

In [2]:
load_dotenv()

True

In [3]:
# Get the key from env variable
groq_key = os.getenv("GROQ_API_KEY")

In [4]:
model = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key
)

In [5]:
# define the state of the blog
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [6]:
# define function to generate a blog outline 

def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state["title"]

    # form a prompt 
    prompt = f"Create a blog outline for the following title: {title}"

    # ask that question to the LLM using the OpenAI-compatible SDK
    response = model.chat.completions.create(
        model="llama3-70b-8192",  # or any other supported Groq model
        messages=[{"role": "user", "content": prompt}]
    )

    # extract the outline
    outline = response.choices[0].message.content

    # update the state with the outline 
    state["outline"] = outline

    return state

In [7]:
# define function to generate a blog 

def create_blog(state: BlogState) -> BlogState:

    title = state["title"]
    outline = state["outline"]

    prompt = f"Create a blog for the following title: {title} and outline: {outline}"

    # ask that question to the LLM using the OpenAI-compatible SDK
    response = model.chat.completions.create(
        model="llama3-70b-8192",  # or any other supported Groq model
        messages=[{"role": "user", "content": prompt}]
    )

    # extract the outline
    content = response.choices[0].message.content

    state["content"] = content

    return state 

In [8]:
# create our graph 
graph = StateGraph(BlogState)

# create nodes 
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)

# create edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)

# compile the graph
workflow = graph.compile()


In [9]:
intial_state = {'title': 'World in 2030'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'World in 2030', 'outline': 'Here is a proposed outline for a blog post on the title "World in 2030":\n\n**I. Introduction**\n\n* Brief overview of the rapid pace of technological advancements and global changes\n* Thesis statement: By 2030, the world will undergo significant transformations in various aspects of life, from technology and economy to environment and society.\n\n**II. Technology and Innovation**\n\n* Predictions for artificial intelligence, machine learning, and robotics in 2030\n\t+ Impact on industries such as healthcare, finance, and education\n\t+ Automation of jobs and the future of work\n* Advances in renewable energy and transportation\n\t+ Electric cars and hyperloops: the future of transportation\n\t+ Solar and wind energy: powering homes and cities\n* The rise of virtual and augmented reality\n\t+ Changing the way we interact, work, and play\n\n**III. Economy and Business**\n\n* Global economic trends and predictions\n\t+ Rise of emerging markets and 

In [10]:
print(final_state['outline'])

Here is a proposed outline for a blog post on the title "World in 2030":

**I. Introduction**

* Brief overview of the rapid pace of technological advancements and global changes
* Thesis statement: By 2030, the world will undergo significant transformations in various aspects of life, from technology and economy to environment and society.

**II. Technology and Innovation**

* Predictions for artificial intelligence, machine learning, and robotics in 2030
	+ Impact on industries such as healthcare, finance, and education
	+ Automation of jobs and the future of work
* Advances in renewable energy and transportation
	+ Electric cars and hyperloops: the future of transportation
	+ Solar and wind energy: powering homes and cities
* The rise of virtual and augmented reality
	+ Changing the way we interact, work, and play

**III. Economy and Business**

* Global economic trends and predictions
	+ Rise of emerging markets and decline of traditional powers
	+ Impact of automation on employmen

In [11]:
print(final_state['content'])

**The World in 2030: A Future of Transformation and Opportunity**

As we stand at the threshold of a new decade, it's clear that the world is changing at an unprecedented pace. Technological advancements, global shifts, and emerging trends are redefining the way we live, work, and interact with each other. In this blog post, we'll explore the transformative changes that are expected to shape the world by 2030, from technological innovations to environmental concerns, and from economic shifts to societal evolutions.

**Rapid Pace of Change**

In just the past few decades, we've witnessed incredible advancements in technology, healthcare, and communication. The internet, social media, and mobile devices have revolutionized the way we connect, access information, and conduct business. However, the pace of change is only accelerating, and by 2030, we can expect even more profound transformations that will reshape our world.

**Technology and Innovation**

Artificial intelligence, machine l